In [1]:
import numpy as np
import time 
import matplotlib.image as image
import matplotlib.pyplot as plt
import multiprocessing
from squanch import *

In [2]:
class Alice(Agent):
    '''Alice sends states to Bob and receives results'''
    def run(self):
        measurements = []
        for i, qsys in enumerate(self.qstream):
            q = qsys.qubit(0)
            if (i % 2 == 0): X(q) # flip even qubits
            self.qsend(bob, q)
            result = self.crecv(bob)
            measurements.append(result)
        self.output({"meas": measurements,"t": self.time})
        
class Bob(Agent):
    '''Bob measures Alics states and sends outcomes'''
    def run(self):
        for _ in self.qstream:
            q = self.qrecv(alice)
            self.csend(alice, q.measure())
        self.output({"t": self.time})

# Set up QStream and Agent instances
multiprocessing.set_start_method("fork", force=True)
qstream = QStream(1, 10)
out = Agent.shared_output()
alice = Alice(qstream, out)
bob = Bob(qstream, out)
# Connect agents with classical and quantum channels
alice.qconnect(bob)
alice.cconnect(bob)
# Run the simulation
Simulation(alice, bob).run()
print("Measurements: {}".format(out["Alice"]["meas"]))

Alice:   0%|          | 0/10 [00:00<?, ?it/s]

Bob:   0%|          | 0/10 [00:00<?, ?it/s]

Measurements: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [3]:
class SimpleError(QError):
    def apply(self, qubit):
        '''Flip the qubit with 50% probability'''
        if np.random.rand() < 0.5:
            X(qubit)
        return qubit

class SimpleQChannel(QChannel):
    def __init__(self, *args, **kwargs):
        QChannel.__init__(self, *args, **kwargs)
        self.errors = [SimpleError(self)]

In [4]:
# Set up QStream and Agent instances
multiprocessing.set_start_method("fork", force=True)
qstream = QStream(1, 10)
out = Agent.shared_output()
alice = Alice(qstream, out)
bob = Bob(qstream, out)
alice.pulse_length = 10e-9 # 10ns pulse length
bob.pulse_length = 10e-9
# Connect agents with new model, specify 1km length
alice.qconnect(bob, SimpleQChannel, length=1.0)
alice.cconnect(bob, length=1.0)

In [5]:
# Run the simulation
Simulation(alice, bob).run()
alice_out, bob_out = out["Alice"], out["Bob"]
print("Alices time: {}".format(alice_out["t"]))
print("Bobs time: {}".format(bob_out["t"]))
print("Measurements: {}".format(alice_out["meas"]))

Alice:   0%|          | 0/10 [00:00<?, ?it/s]

Bob:   0%|          | 0/10 [00:00<?, ?it/s]

Alices time: 8.761114076050704e-05
Bobs time: 8.427558372248169e-05
Measurements: [0, 1, 0, 0, 1, 0, 0, 1, 1, 1]


In [6]:
import sys
sys.getsizeof(1)

28